In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd
import numpy as np
import calendar
import datetime
import re
import os

def filter_data(input_file_path,keyword):
    file = os.listdir(input_file_path)[0]
    dataFrame= pd.read_csv(os.path.join(input_file_path, file))
    # select rows containing text "Affirm"
    data_filtered = dataFrame.loc[dataFrame['DESCRIPTION'].str.contains(keyword),['DATE', 'DESCRIPTION', 'RECEIVED']]
    # data_filtered.to_csv(os.path.join(output_file_path, file), index = False)

    #split the decription
    new =data_filtered.DESCRIPTION.str.split(expand=True)
    data_filtered['DEPOSIT_ID']=new[15]
    data_filtered.drop(columns =["DESCRIPTION"], inplace = True)
    data_filtered['DESCRIPTION']=new[3]+' '+new[4]

    # split the deposit id
    new_id=data_filtered.DEPOSIT_ID.str.split(":",expand=True)
    data_filtered.drop(columns =["DEPOSIT_ID"], inplace = True)
    data_filtered['DEPOSIT_ID']=new_id[1]
    return data_filtered

In [19]:
def merge_filtered(res_data,affirm_report):
   #read the affirm report file
   data_report=pd.read_csv(affirm_report)
   data_report.rename(columns = {'deposit_id':'DEPOSIT_ID'}, inplace = True)
   #combine two csv files based on DEPOSIT ID
   df = pd.merge(res_data, data_report,on="DEPOSIT_ID")
  #  print(df.to_string())
   df.rename(columns = {'RECEIVED':'BANK AMOUNT'}, inplace = True)
   df.rename(columns = {'order_id':'ORDER'}, inplace = True)
   df['FEE']=df['fees']+df['txn_fees']
   df['ORDER']=df['ORDER'].astype(str)
   return df


In [21]:
def merge_order(data_report,order_report_table):
    report_table=pd.read_csv(order_report_table)   #read the order report table
    report_table.rename(columns = {'Order_Number':'ORDER'}, inplace = True)
    #covert the object type to float
    report_table['Item_Price']=report_table['Item_Price'].apply(lambda x: float(x.split()[0].replace(',', '')))
    report_table['Item_Discount']=report_table['Item_Discount'].apply(lambda x: float(x.split()[0].replace(',', '')))
    
    #combine report table to df
    data_table=pd.merge(data_report,report_table,on='ORDER')
    data_table.rename(columns = {'Item_Price':'PRICE'}, inplace = True) # rename columns
    data_table.rename(columns = {'Item_Discount':'DISCOUNT'}, inplace = True) # rename columns
    data_table.rename(columns = {'Order_Date':'QBO DATE'}, inplace = True) # rename columns
    data_table['QBO Description']=data_table['ORDER']
    data_table['INCOME']=data_table['PRICE']

    #add new columns
    data_table['PRICE']=data_table['PRICE'].astype(float)
    data_table['DISCOUNT']=data_table['DISCOUNT'].astype(float)
    data_table['NET AMOUNT']=data_table['PRICE']+data_table['FEE']+data_table['DISCOUNT']
    
    #add a new columns check by comparing two columns
    data_table['CHECK']=np.where((data_table['NET AMOUNT'] == data_table['total_settled']), 'Correct', 'Incorrect')
    # print(data_table.to_string())

    #change the datatype to int
    data_table['ORDER']=data_table['ORDER'].astype(int)

    return data_table

In [ ]:
data=filter_data('/content/drive/MyDrive/Colab Notebooks/input_file_path','AFFIRM')
res_data=merge_filtered(data,'/content/drive/MyDrive/Affirm Testing/Affirm_Report_0123.csv')
merge_order=merge_order(res_data,'/content/drive/MyDrive/Affirm Testing/order_report_Jan22_Jan23.csv')
print(merge_order)